# Un problema de clasificación con k vecinos más cercanos: Credit
*Objetivo:* Construir un asistente financiero para predecir el pago de un crédito

**NOTA:** Tarea de clasificación.

In [1]:
import pandas as pd  # Para manipular los dataframes
import numpy as np   # Para operar las matrices de datos y manejo de vectores

import seaborn as sns               # Librerias de gráficos
import matplotlib.pyplot as plt     # Librerias de gráficos

import warnings
warnings.filterwarnings("ignore")

from sklearn.neighbors import KNeighborsClassifier # Librería de algoritmos de machine learning
                                                   # Utilizamos k-vecinos para clasificación 

from sklearn.feature_selection import SelectKBest, f_classif  # Para seleccionar los "mejores" atributos 
                                                              # f_classif solo para atributos continuos y chi2 para atributos categóricos

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler  #Para escalar datos
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report  # PAra reportar las métricas
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold,StratifiedKFold, cross_val_score, cross_validate  
from sklearn.model_selection import GridSearchCV   # Calibración de hiperparámetros
from sklearn.pipeline import make_pipeline
from scipy import stats

In [2]:
# Cargar los datos 
datos=pd.read_csv("C:/Users/migue/Documents/DSML_5_Modulo1/datos/loan3000.csv")

# Datos y tipos de datos:
**ID**: Identificador del cliente, para nuestro análisis esta variable la descartar

**outcome:**  (target) Resultado del crédito (paid off: crédito pagado, default: Credito con retraso o no pagado) (categórica)

**purpose_:** (categórica) Propósito del crédito

**dti:** (continua) Indice de ingreso a deuda. dti mide el porcentaje de los ingresos mensuales brutos de una persona que se destina al pago de deudas mensuales.

**borrower_score:** (continua) Equivalente al score del buro de crédito

**payment_inc_ratio:** (continua) El cociente entre el pago mensual del crédito y el ingreso mensual


In [3]:
datos.head()

,ID,outcome,purpose_,dti,borrower_score,payment_inc_ratio
0,32109,paid off,debt_consolidation,21.23,0.40,5.11135
1,16982,default,credit_card,15.49,0.40,5.43165
2,25335,paid off,debt_consolidation,27.30,0.70,9.23003
3,34580,paid off,major_purchase,21.11,0.40,2.33482
4,14424,default,debt_consolidation,16.46,0.45,12.10320


# Paso 0. Preprocesamiento
En esta etapa: codificación de los datos, selección de atributos, escalamiento de los datos,...

In [4]:
# Descartamos el ID
datos=datos.drop("ID", axis=1)

In [5]:
# Revisar la base datos
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   outcome            3000 non-null   object 
 1   purpose_           3000 non-null   object 
 2   dti                3000 non-null   float64
 3   borrower_score     3000 non-null   float64
 4   payment_inc_ratio  3000 non-null   float64
dtypes: float64(3), object(2)
memory usage: 117.3+ KB


In [6]:
datos.describe()

,dti,borrower_score,payment_inc_ratio
count,3000.000000,3000.000000,3000.000000
mean,16.120103,0.502333,8.137902
std,7.590890,0.128297,4.296310
min,0.000000,0.100000,0.221906
25%,10.362500,0.400000,4.758135
50%,15.985000,0.500000,7.588420
75%,21.360000,0.600000,11.091500
max,35.270000,0.900000,33.630900


### Para ver el efecto del escalamiento de los datos

In [7]:
escala=RobustScaler()
X=escala.fit_transform(datos[["dti","borrower_score","payment_inc_ratio"]])
X=pd.DataFrame(X, columns=["dti","borrower_score","payment_inc_ratio"])
X.describe()

,dti,borrower_score,payment_inc_ratio
count,3.000000e+03,3000.000000,3000.000000
mean,1.228491e-02,0.011667,0.086760
std,6.902378e-01,0.641483,0.678361
min,-1.453512e+00,-2.000000,-1.163128
25%,-5.112526e-01,-0.500000,-0.446885
50%,8.077306e-17,0.000000,0.000000
75%,4.887474e-01,0.500000,0.553115
max,1.753580e+00,2.000000,4.111950


...Continuamos con el análisis

In [8]:
# Los datos están balanceados? SI
datos["outcome"].value_counts(normalize=True)

outcome
paid off    0.518333
default     0.481667
Name: proportion, dtype: float64

In [9]:
# Todo lo que sea letras lo debemos convertir a números, en este caso serías dos variables categóricas:
# outcome y purpose_, aunque esta ultima la vamos a abordar de manera diferente

In [10]:
# Nueva linea para codificar las categorías del target  
equivalencia={"paid off":0,"default":1}  # Hacemos un diccionario para las equivalencias de "letras" a números
datos["outcome"]=datos["outcome"].map(equivalencia)  # Reescribimos las columna de diagnóstico
datos["outcome"]=datos["outcome"].astype("category") 

In [11]:
datos.head()

,outcome,purpose_,dti,borrower_score,payment_inc_ratio
0,0,debt_consolidation,21.23,0.40,5.11135
1,1,credit_card,15.49,0.40,5.43165
2,0,debt_consolidation,27.30,0.70,9.23003
3,0,major_purchase,21.11,0.40,2.33482
4,1,debt_consolidation,16.46,0.45,12.10320


sns.pairplot(datos, hue="outcome")

In [12]:
# Cómo trabajamos con una covariable o atributo cualitativo? Creando variable dummies
datos["purpose_"].value_counts(normalize=True)*100

purpose_
debt_consolidation    57.933333
credit_card           17.500000
other                 10.600000
home_improvement       6.200000
major_purchase         3.833333
small_business         2.933333
medical                1.000000
Name: proportion, dtype: float64

## ¿Cuáles son los atributos relevantes para incluir en el modelo?

In [13]:
# El propósito del crédito está asociado con el outcome?
from scipy.stats import chi2_contingency
tabla = pd.crosstab(datos["purpose_"], datos["outcome"])
#tabla
resultado=chi2_contingency(tabla)
resultado
# Prueba de hipótesis:
# Ho: purpose_ y outcome son independientes
# H1: Existe asociación entre las variables purpose_ y outcome

Chi2ContingencyResult(statistic=36.54917585703614, pvalue=2.1555343121927533e-06, dof=6, expected_freq=array([[272.125     , 252.875     ],
       [900.86333333, 837.13666667],
       [ 96.41      ,  89.59      ],
       [ 59.60833333,  55.39166667],
       [ 15.55      ,  14.45      ],
       [164.83      , 153.17      ],
       [ 45.61333333,  42.38666667]]))

In [14]:
# Como el p-value<0.05 la conclusión es que existe asociación entre purpose y outcome 

In [15]:
resultado.expected_freq  # Bajo Ho es cierta

array([[272.125     , 252.875     ],
       [900.86333333, 837.13666667],
       [ 96.41      ,  89.59      ],
       [ 59.60833333,  55.39166667],
       [ 15.55      ,  14.45      ],
       [164.83      , 153.17      ],
       [ 45.61333333,  42.38666667]])

In [16]:
tabla

outcome,0,1
purpose_,,
credit_card,318,207
debt_consolidation,876,862
home_improvement,106,80
major_purchase,66,49
medical,11,19
other,146,172
small_business,32,56


### Creamos las dummies para purpose_

In [17]:
datos=pd.get_dummies(datos,columns=["purpose_"])
datos.head()

,outcome,dti,borrower_score,payment_inc_ratio,purpose__credit_card,purpose__debt_consolidation,purpose__home_improvement,purpose__major_purchase,purpose__medical,purpose__other,purpose__small_business
0,0,21.23,0.40,5.11135,False,True,False,False,False,False,False
1,1,15.49,0.40,5.43165,True,False,False,False,False,False,False
2,0,27.30,0.70,9.23003,False,True,False,False,False,False,False
3,0,21.11,0.40,2.33482,False,False,False,True,False,False,False
4,1,16.46,0.45,12.10320,False,True,False,False,False,False,False


**NOTA:** Selección de los atributos. En dos etapas, una para las continuas y la segunda para las categóricas

In [18]:
# Selección de atributos
from sklearn.feature_selection import SelectKBest, f_classif, chi2
y=datos["outcome"]
X_continuas=datos[['dti', 'borrower_score', 'payment_inc_ratio']]
X_categoricas= datos[['purpose__credit_card', 'purpose__debt_consolidation','purpose__home_improvement', 'purpose__major_purchase','purpose__medical', 'purpose__other', 'purpose__small_business']]
selector=SelectKBest(score_func=f_classif,k=3) 
selector.fit_transform(X_continuas,y)                     # Aquí ya hace la tarea de seleccionar los mejores atributos

# Para que me diga cuáles son los k mejores atributos ejecutamos
X_continuas.columns[selector.get_support()]

Index(['dti', 'borrower_score', 'payment_inc_ratio'], dtype='object')

In [19]:
selector=SelectKBest(score_func=chi2,k=3) 
selector.fit_transform(X_categoricas,y)                     # Aquí ya hace la tarea de seleccionar los mejores atributos

# Para que me diga cuáles son los k mejores atributos ejecutamos
X_categoricas.columns[selector.get_support()]

Index(['purpose__credit_card', 'purpose__other', 'purpose__small_business'], dtype='object')

In [20]:
# Seleccionamos los atributos y el target
y=datos["outcome"]      # target
X=datos[["borrower_score","payment_inc_ratio","dti","purpose__credit_card",'purpose__other',"purpose__small_business"]] #atributos
X.head()

,borrower_score,payment_inc_ratio,dti,purpose__credit_card,purpose__other,purpose__small_business
0,0.40,5.11135,21.23,False,False,False
1,0.40,5.43165,15.49,True,False,False
2,0.70,9.23003,27.30,False,False,False
3,0.40,2.33482,21.11,False,False,False
4,0.45,12.10320,16.46,False,False,False


In [21]:
X.shape

(3000, 6)

# Paso 1: Definir el modelo

In [22]:
modelo=KNeighborsClassifier()  # Definimos el modelo

### Paso 1.1. Métrica del desempeño del modelo, antes de seleccionar el mejor modelo
¿Cuál es el número adecuado de vecinos que debemos emplear en el método?. Si ya lo supiéramos
por ejemplo k

In [23]:
# Estandarizar las continuas y dejar pasar las demás
# Es decir, solo aplicamos transformación a las variables continuas
from sklearn.compose import make_column_transformer              # Dar tratamiento diferenciado a cada columna
colum_trans= make_column_transformer((StandardScaler(), ["borrower_score","payment_inc_ratio","dti"]),remainder='passthrough')

In [24]:
from sklearn.pipeline import make_pipeline

# Cómo incluimos el escalamiento de los datos en una validación cruzada:
# i) dividir al conjunto en k-folds
# ii) utilizamos k-1 folds de train y el fold restante de prueba
# iii) (pipe) vamos entrenar un escalamiento diferenciado, una estandarización en las continuas y dejamos pasar las categóricas
# iv) escalamos los datos de train con el escalamiento elegido (i.e transformamos los datos de entrenamiento)
# v) Utilizando los datos transformados entrenamos el modelo : KNeighborsClassifier(n_neighbors=7)
# vi) escalamos los datos de prueba (el fold reservado) empleando el escalamiento entrenado en iii)
# vii) hacemos la predicciones con el modelo entrenado y los datos de prueba transformados
# viii) Medimos el desempeño preditivo del modelo empleando la métrica definida en scoring
# ix) Reportamos la métrica
k=27
pipe=make_pipeline(colum_trans,KNeighborsClassifier(n_neighbors=k))
scores=cross_val_score(pipe,X,y,cv=10,scoring="precision",n_jobs=-1)
print(f"El desempeño promedio del modelo con {k} vecinos es: {scores.mean()*100:.2f}% de precision")  

El desempeño promedio del modelo con 27 vecinos es: 61.27% de precision


## ¿Cuál es el mejor valor de k?

# Paso 2: Calibrar el modelo, para seleccionar el mejor modelo

In [25]:
# Paso i: Definir el espacio parametral
espacio_param={"kneighborsclassifier__n_neighbors":np.arange(1,60,2),"kneighborsclassifier__p":[1,2]}
# Paso ii: Definir el flujo de procesamiento: pipe
pipe=make_pipeline(colum_trans,KNeighborsClassifier())  # En palabras: Primero escalar los datos y luego entrenar el modelo
# Paso iii) Definir el diseño de validación cruzada
cv_diseño=RepeatedKFold(n_splits=10,n_repeats=5)
# Definimos la rejilla de búsqueda
rejilla=GridSearchCV(pipe,param_grid=espacio_param,scoring="precision",cv=cv_diseño,n_jobs=-1)
rejilla.fit(X,y)
print("La mejor configuración de parámetros es:", rejilla.best_params_)
print("La exactitud de la mejor configuración:", rejilla.best_score_)

La mejor configuración de parámetros es: {'kneighborsclassifier__n_neighbors': 41, 'kneighborsclassifier__p': 2}
La exactitud de la mejor configuración: 0.6214265475855195


**OJO:** La métrica puede estar sobreestimada... necesitamos hacer una validación cruzada anidada para tener una métrica más realista

### Validación cruzada anidada

In [26]:
# Paso i: Definir el espacio parametral
espacio_param={"kneighborsclassifier__n_neighbors":np.arange(1,60,2),"kneighborsclassifier__p":[1,2]}
# Paso ii: Definir el flujo de procesamiento: pipe
pipe=make_pipeline(colum_trans,KNeighborsClassifier())  # En palabras: Primero escalar los datos y luego entrenar el modelo
# Paso iii) Definir el diseño de validación cruzada interna y externa
cv_interna=RepeatedKFold(n_splits=10,n_repeats=5)
cv_externa=StratifiedKFold(n_splits=10)
# Definimos la rejilla de búesqueda
rejilla=GridSearchCV(pipe,param_grid=espacio_param,scoring="precision",cv=cv_interna,n_jobs=-1)
scores=cross_val_score(rejilla,X,y,scoring="precision",cv=cv_externa,n_jobs=-1)

In [33]:
from scipy import stats
#print("Exactitud promedio:", scores.mean())
#print("Error estándar de la media muestral:",stats.sem(scores))              # SE (Error Estándar) = Desviación Estándar Muestral / raiz_cuadrada(tamaño de la media muestral)
print(f"La precisión promedio es: {scores.mean():.4f} +/- {stats.sem(scores):.4f}") 

La precisión promedio es: 0.6156 +/- 0.0076


# Etapa final del entrenamiento

In [34]:
# IMPORTANTE: Para entrenar el modelo definitivo o final, vamos entrenarlo empleando TODOS* (descartando sspg) los datos
# Paso 1. Etapa de escalamiento de los datos
escala=colum_trans
X_std=escala.fit_transform(X)   # El métod escala me va a servir para hacer predicciones futuras
# Paso 2: Entrenar el algoritmo
modelo_final=KNeighborsClassifier(n_neighbors=41,p=2)
modelo_final.fit(X_std,y)

KNeighborsClassifier(n_neighbors=41)

In [35]:
# Este modelo final ya lo pongo en producción... 
# Sabemos que el modelo tiene una precision del 0.6156 +/- 0.0076

### Predicciones en datos nuevos

In [36]:

datos_nuevos=pd.DataFrame({"borrower_score":[0.2,0.5,0.7],
                           "payment_inc_ratio":[10,25,7],
                           "dti":[33,10,5],
                           "purpose__credit_card":[1,0,0],
                           'purpose__other':[0,1,1],
                           "purpose__small_business":[0,0,0]})
datos_nuevos

,borrower_score,payment_inc_ratio,dti,purpose__credit_card,purpose__other,purpose__small_business
0,0.2,10,33,1,0,0
1,0.5,25,10,0,1,0
2,0.7,7,5,0,1,0


In [37]:
#Escalamos los datos nuevos
datos_nuevos_std=escala.transform(datos_nuevos) 
y_pred=modelo_final.predict(datos_nuevos_std)

In [38]:
# Para obtener los diagnósticos en la denominación original
equivalencia_inverso={0:'paid off', 1:'Default'}
nuevas=pd.Series(y_pred).map(equivalencia_inverso)
datos_nuevos["outcome*"]=nuevas
datos_nuevos

,borrower_score,payment_inc_ratio,dti,purpose__credit_card,purpose__other,purpose__small_business,outcome*
0,0.2,10,33,1,0,0,Default
1,0.5,25,10,0,1,0,Default
2,0.7,7,5,0,1,0,paid off


# Nota: 
En los dos primeros casos estamos 61.56% seguros de que son clientes que van a caer en default